In [1]:

import nlpaug.augmenter.word as naw
aug = naw.ContextualWordEmbsAug(model_path='klue/bert-base', action='insert')

The following layers were not sharded: cls.predictions.transform.LayerNorm.weight, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.attention.self.query.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, cls.predictions.transform.LayerNorm.bias, cls.predictions.decoder.bias, bert.encoder.layer.*.attention.self.value.weight, cls.predictions.bias, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.intermediate.dense.weight, cls.predictions.transform.dense.bias, bert.embeddings.LayerNorm.weight, cls.predictions.decoder.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.attention.output.dense.weight, cls.predictions.transform.dense.weight, bert.encoder.laye

In [ ]:
import json
with open('통합_데이터셋.json', "r", encoding='utf-8') as f:
  data = json.load(f)

augmented_data = []
k=0
for i in data:
  k+=1
  q = i['question']
  a = i['answer']

  for j in range(2):
    new_q = aug.augment(q)
    augmented_data.append({"question":new_q,"answer":a})
  augmented_data.append({"question" : q, "answer":a})
  print(f'데이터 {k}개 증강 완료')

with open("augmented_data.json", "w", encoding="utf-8") as f:
  json.dump(augmented_data, f, ensure_ascii=False, indent=2)

In [29]:
aug2 = naw.RandomWordAug(action='delete', aug_p=0.2, stopwords=['강아지'])

text = "우리 강아지가 밥을 안 먹어요"
augmented = aug2.augment(text)
augmented

['우리 밥을 안 먹어요']

In [30]:
aug.augment(augmented)

['왜 우리 어제 밥을 안 먹어요']

In [31]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_1300/3596193378.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [33]:
text = "개가 물 많이 마시고 게워내요"

In [35]:
result = EDA(text)
print(result)

['많이 물 개가 마시고 게워내요', '개가 물 많이 물 마시고 게워내요', '개가 물 물 많이 마시고 게워내요', '게워내요 물 많이 마시고 개가', '개가 물 많이 게워내요', '개가 많이 마시고 게워내요', '개가 물 많이 게워내요', '개가 놈 많이 마시고 게워내요', '개가 물 많이 마시고 물 게워내요', '개가 물 많이 마시고 게워내요']


In [1]:
import random
from jamo import h2j, j2h, is_jamo

def typo_korean(text, p=0.05):
    result = ""
    for ch in text:
        # 한글 자모 분리
        try:
            jamos = list(h2j(ch))
        except:
            result += ch
            continue

        new_jamos = []
        for j in jamos:
            if is_jamo(j) and random.random() < p:
                # 자모 교체
                new_jamos.append(random.choice(['ㄱ','ㄴ','ㄷ','ㅂ','ㅅ','ㅈ','ㅊ','ㅎ','ㅁ']))
            else:
                new_jamos.append(j)
        
        try:
            result += j2h(*new_jamos)
        except Exception:
            result += ch
    return result

print(typo_korean("우리 강아지가 밥을 안 먹어요"))


우리 강아히가 밥을 안 먹어요


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
with open('augmented_data2.json', "r", encoding='utf-8') as f:
  data = json.load(f)

augmented_data = []
k=0
for i in data:
  k+=1
  q = i['question']
  a = i['answer']

  new_q = typo_korean(q)
  augmented_data.append({"question":new_q,"answer":a})
  augmented_data.append({"question" : q, "answer":a})
  print(f'데이터 {k}개 증강 완료')

with open("augmented_data3.json", "w", encoding="utf-8") as f:
  json.dump(augmented_data, f, ensure_ascii=False, indent=2)

데이터 1개 증강 완료
데이터 2개 증강 완료
데이터 3개 증강 완료
데이터 4개 증강 완료
데이터 5개 증강 완료
데이터 6개 증강 완료
데이터 7개 증강 완료
데이터 8개 증강 완료
데이터 9개 증강 완료
데이터 10개 증강 완료
데이터 11개 증강 완료
데이터 12개 증강 완료
데이터 13개 증강 완료
데이터 14개 증강 완료
데이터 15개 증강 완료
데이터 16개 증강 완료
데이터 17개 증강 완료
데이터 18개 증강 완료
데이터 19개 증강 완료
데이터 20개 증강 완료
데이터 21개 증강 완료
데이터 22개 증강 완료
데이터 23개 증강 완료
데이터 24개 증강 완료
데이터 25개 증강 완료
데이터 26개 증강 완료
데이터 27개 증강 완료
데이터 28개 증강 완료
데이터 29개 증강 완료
데이터 30개 증강 완료
데이터 31개 증강 완료
데이터 32개 증강 완료
데이터 33개 증강 완료
데이터 34개 증강 완료
데이터 35개 증강 완료
데이터 36개 증강 완료
데이터 37개 증강 완료
데이터 38개 증강 완료
데이터 39개 증강 완료
데이터 40개 증강 완료
데이터 41개 증강 완료
데이터 42개 증강 완료
데이터 43개 증강 완료
데이터 44개 증강 완료
데이터 45개 증강 완료
데이터 46개 증강 완료
데이터 47개 증강 완료
데이터 48개 증강 완료
데이터 49개 증강 완료
데이터 50개 증강 완료
데이터 51개 증강 완료
데이터 52개 증강 완료
데이터 53개 증강 완료
데이터 54개 증강 완료
데이터 55개 증강 완료
데이터 56개 증강 완료
데이터 57개 증강 완료
데이터 58개 증강 완료
데이터 59개 증강 완료
데이터 60개 증강 완료
데이터 61개 증강 완료
데이터 62개 증강 완료
데이터 63개 증강 완료
데이터 64개 증강 완료
데이터 65개 증강 완료
데이터 66개 증강 완료
데이터 67개 증강 완료
데이터 68개 증강 완료
데이터 69개 증강 완료
데이터 70개 증강 완료
데이터 71개 증강 완료
데이터 72개 증강 완료
데

Could not parse jamo: U+314e
Could not parse jamo: U+3145
Could not parse jamo: U+314e
Could not parse jamo: U+314e
Could not parse jamo: U+314a
Could not parse jamo: U+3137
Could not parse jamo: U+3137
Could not parse jamo: U+3141
Could not parse jamo: U+314e
Could not parse jamo: U+3137
Could not parse jamo: U+3141
Could not parse jamo: U+3141
Could not parse jamo: U+3134
Could not parse jamo: U+3134
Could not parse jamo: U+3142
Could not parse jamo: U+3148
Could not parse jamo: U+3148
Could not parse jamo: U+3137
Could not parse jamo: U+3142
Could not parse jamo: U+3148
Could not parse jamo: U+3145
Could not parse jamo: U+3141
Could not parse jamo: U+314a
Could not parse jamo: U+3137
Could not parse jamo: U+3142
Could not parse jamo: U+3134
Could not parse jamo: U+3134
Could not parse jamo: U+3137
Could not parse jamo: U+3134
Could not parse jamo: U+314e
Could not parse jamo: U+3148
Could not parse jamo: U+3148
Could not parse jamo: U+3134
Could not parse jamo: U+314e
Could not pars

데이터 1345개 증강 완료
데이터 1346개 증강 완료
데이터 1347개 증강 완료
데이터 1348개 증강 완료
데이터 1349개 증강 완료
데이터 1350개 증강 완료
데이터 1351개 증강 완료
데이터 1352개 증강 완료
데이터 1353개 증강 완료
데이터 1354개 증강 완료
데이터 1355개 증강 완료
데이터 1356개 증강 완료
데이터 1357개 증강 완료
데이터 1358개 증강 완료
데이터 1359개 증강 완료
데이터 1360개 증강 완료
데이터 1361개 증강 완료
데이터 1362개 증강 완료
데이터 1363개 증강 완료
데이터 1364개 증강 완료
데이터 1365개 증강 완료
데이터 1366개 증강 완료
데이터 1367개 증강 완료
데이터 1368개 증강 완료
데이터 1369개 증강 완료
데이터 1370개 증강 완료
데이터 1371개 증강 완료
데이터 1372개 증강 완료
데이터 1373개 증강 완료
데이터 1374개 증강 완료
데이터 1375개 증강 완료
데이터 1376개 증강 완료
데이터 1377개 증강 완료
데이터 1378개 증강 완료
데이터 1379개 증강 완료
데이터 1380개 증강 완료
데이터 1381개 증강 완료
데이터 1382개 증강 완료
데이터 1383개 증강 완료
데이터 1384개 증강 완료
데이터 1385개 증강 완료
데이터 1386개 증강 완료
데이터 1387개 증강 완료
데이터 1388개 증강 완료
데이터 1389개 증강 완료
데이터 1390개 증강 완료
데이터 1391개 증강 완료
데이터 1392개 증강 완료
데이터 1393개 증강 완료
데이터 1394개 증강 완료
데이터 1395개 증강 완료
데이터 1396개 증강 완료
데이터 1397개 증강 완료
데이터 1398개 증강 완료
데이터 1399개 증강 완료
데이터 1400개 증강 완료
데이터 1401개 증강 완료
데이터 1402개 증강 완료
데이터 1403개 증강 완료
데이터 1404개 증강 완료
데이터 1405개 증강 완료
데이터 1406개 증강 완료
데이터 1407

Could not parse jamo: U+3141
Could not parse jamo: U+314e
Could not parse jamo: U+3141
Could not parse jamo: U+3145
Could not parse jamo: U+3148
Could not parse jamo: U+314e
Could not parse jamo: U+3134
Could not parse jamo: U+314a
Could not parse jamo: U+3148
Could not parse jamo: U+3141
Could not parse jamo: U+3131
Could not parse jamo: U+3145
Could not parse jamo: U+314a
Could not parse jamo: U+3142
Could not parse jamo: U+3145
Could not parse jamo: U+3145
Could not parse jamo: U+3142
Could not parse jamo: U+3141
Could not parse jamo: U+3145
Could not parse jamo: U+3137
Could not parse jamo: U+3145
Could not parse jamo: U+3137
Could not parse jamo: U+3145
Could not parse jamo: U+3148
Could not parse jamo: U+3142
Could not parse jamo: U+3134
Could not parse jamo: U+314a
Could not parse jamo: U+3148
Could not parse jamo: U+314a
Could not parse jamo: U+3142
Could not parse jamo: U+3142
Could not parse jamo: U+3141
Could not parse jamo: U+314a
Could not parse jamo: U+3142
Could not pars

In [1]:
from googletrans import Translator
trans = Translator()

result = trans.translate("저희 강아지가 토해요", dest="en", src='ko')
print(result.text)
result2 = trans.translate(result.text, dest='ko',src='en')
print(result2.text)

My dog ​​is throwing up
우리 강아지가 토해요


In [2]:
import re

def clean_text(t):
    return re.sub(r'[^\w\s가-힣]', '', t)
import time, random
import json
from langdetect import detect 
from deep_translator import GoogleTranslator
with open('통합_데이터셋.json', "r", encoding='utf-8') as f:
  data = json.load(f)

augmented_data = []
k=0
for i in data:
  k+=1
  q = i['question']
  a = i['answer']

  if detect(q) == 'ko':
    q = clean_text(q)
    time.sleep(random.uniform(0.5, 1.5))
    new_q = GoogleTranslator(source='ko',target='en').translate(q)
    trans_q = new_q
  else :
    trans_q = q
  
  time.sleep(random.uniform(0.5, 1.5))
  new_q_ko = GoogleTranslator(source='en',target='ko').translate(trans_q)
  print(f'데이터 {k}개 번역')
  augmented_data.append({"question":new_q_ko,"answer":a})
  augmented_data.append({"question" : q, "answer":a})
  

데이터 1개 번역
데이터 2개 번역
데이터 3개 번역
데이터 4개 번역
데이터 5개 번역
데이터 6개 번역
데이터 7개 번역
데이터 8개 번역
데이터 9개 번역
데이터 10개 번역
데이터 11개 번역
데이터 12개 번역
데이터 13개 번역
데이터 14개 번역
데이터 15개 번역
데이터 16개 번역
데이터 17개 번역
데이터 18개 번역
데이터 19개 번역
데이터 20개 번역
데이터 21개 번역
데이터 22개 번역
데이터 23개 번역
데이터 24개 번역
데이터 25개 번역
데이터 26개 번역
데이터 27개 번역
데이터 28개 번역
데이터 29개 번역
데이터 30개 번역
데이터 31개 번역
데이터 32개 번역
데이터 33개 번역
데이터 34개 번역
데이터 35개 번역
데이터 36개 번역
데이터 37개 번역
데이터 38개 번역
데이터 39개 번역
데이터 40개 번역
데이터 41개 번역
데이터 42개 번역
데이터 43개 번역
데이터 44개 번역
데이터 45개 번역
데이터 46개 번역
데이터 47개 번역
데이터 48개 번역
데이터 49개 번역
데이터 50개 번역
데이터 51개 번역
데이터 52개 번역
데이터 53개 번역
데이터 54개 번역
데이터 55개 번역
데이터 56개 번역
데이터 57개 번역
데이터 58개 번역
데이터 59개 번역
데이터 60개 번역
데이터 61개 번역
데이터 62개 번역
데이터 63개 번역
데이터 64개 번역
데이터 65개 번역
데이터 66개 번역
데이터 67개 번역
데이터 68개 번역
데이터 69개 번역
데이터 70개 번역
데이터 71개 번역
데이터 72개 번역
데이터 73개 번역
데이터 74개 번역
데이터 75개 번역
데이터 76개 번역
데이터 77개 번역
데이터 78개 번역
데이터 79개 번역
데이터 80개 번역
데이터 81개 번역
데이터 82개 번역
데이터 83개 번역
데이터 84개 번역
데이터 85개 번역
데이터 86개 번역
데이터 87개 번역
데이터 88개 번역
데이터 89개 번역
데이터 90개 번역
데이터 91개 번역
데이터 92개 

In [3]:
with open("augmented_data.json", "w", encoding="utf-8") as f:
  json.dump(augmented_data, f, ensure_ascii=False, indent=2)

In [13]:
pip install deep-translator

Note: you may need to restart the kernel to use updated packages.


In [16]:
from deep_translator import GoogleTranslator

res = GoogleTranslator(source='ko', target='en').translate("우리 강아지가 토해요")
print(res)
res2 = GoogleTranslator(source='en',target='ko').translate(res)
print(res2)

my dog ​​is throwing up
내 개가 토하고 있어요
